In [2]:
import pandas as pd
import numpy as np

In [5]:
case_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-sedelkin/data/KC_case_data .csv', parse_dates=['date'])

In [4]:
# Calculate MAU
case_data['month_index'] = case_data.date.dt.month
case_data.groupby('month_index', as_index=False).agg(mau=('device_id', 'nunique'))

,month_index,mau
0,1,99161
1,2,75032
2,3,74623


In [5]:
# Calculate installs
case_data.loc[case_data.event == 'app_install'].groupby('month_index', as_index=False).agg(installs=('event', 'count'))

,month_index,installs
0,1,80297
1,2,38078
2,3,36222


In [66]:
# Cohorts by install date, CR from install into purchase
cohorts = case_data.loc[case_data.event == 'app_install'][['date', 'device_id']] \
    .rename(columns={'date':'cohort', 'device_id':'device_id'})
cohorts

,cohort,device_id
2515061,2020-01-01,4921563
2515062,2020-01-01,1311583
2515063,2020-01-01,23006325
2515064,2020-01-01,5645652
2515065,2020-01-01,8798567
...,...,...
2669653,2020-03-31,19717968
2669654,2020-03-31,32966665
2669655,2020-03-31,6335964
2669656,2020-03-31,29155826


In [45]:
cohorts.nunique()

cohort           91
device_id    154597
dtype: int64

In [56]:
df = cohorts \
    .merge(case_data.loc[case_data.event == 'purchase'][['date', 'device_id']], on='device_id', how='left') \
    .sort_values('date', ascending=True) \
    .drop_duplicates(subset='device_id')
df

,cohort,device_id,date
11684,2020-01-01,26827118,2020-01-01
1791,2020-01-01,6579389,2020-01-01
1790,2020-01-01,30783948,2020-01-01
8662,2020-01-01,9266476,2020-01-01
8663,2020-01-01,2030633,2020-01-01
...,...,...,...
212358,2020-03-31,2261815,NaT
212359,2020-03-31,19717968,NaT
212360,2020-03-31,32966665,NaT
212361,2020-03-31,6335964,NaT


In [32]:
df.dtypes

cohort       datetime64[ns]
device_id             int64
date         datetime64[ns]
dtype: object

In [57]:
df['purchase_within_week'] = (df.date - df.cohort).dt.days <= 7
df

,cohort,device_id,date,purchase_within_week
11684,2020-01-01,26827118,2020-01-01,True
1791,2020-01-01,6579389,2020-01-01,True
1790,2020-01-01,30783948,2020-01-01,True
8662,2020-01-01,9266476,2020-01-01,True
8663,2020-01-01,2030633,2020-01-01,True
...,...,...,...,...
212358,2020-03-31,2261815,NaT,False
212359,2020-03-31,19717968,NaT,False
212360,2020-03-31,32966665,NaT,False
212361,2020-03-31,6335964,NaT,False


In [67]:
cohorts_cr = df \
    .groupby('cohort', as_index=False) \
    .agg(installs=('device_id', 'nunique'), purchases=('purchase_within_week', 'sum'))

In [68]:
cohorts_cr['cr'] = round((cohorts_cr['purchases'] / cohorts_cr['installs']) * 100, 1)
cohorts_cr.sort_values('cr', ascending=False)

,cohort,installs,purchases,cr
0,2020-01-01,3579,1408,39.3
8,2020-01-09,1424,558,39.2
14,2020-01-15,4310,1650,38.3
13,2020-01-14,5173,1973,38.1
1,2020-01-02,3144,1186,37.7
...,...,...,...,...
77,2020-03-18,1171,235,20.1
88,2020-03-29,1117,223,20.0
81,2020-03-22,1261,251,19.9
87,2020-03-28,1091,209,19.2


In [72]:
case_data.loc[case_data.event == 'app_install'] \
    .groupby('utm_source', as_index=False) \
    .agg(installs=('device_id', 'nunique')) \
    .sort_values('installs', ascending=False)

,utm_source,installs
0,-,32460
6,yandex-direct,29368
2,google_ads,26286
5,vk_ads,23189
3,instagram_ads,20096
1,facebook_ads,13916
4,referal,9282


In [88]:
registered = case_data \
    .loc[case_data.event == 'register'][['device_id', 'date']] \
    .rename(columns={'date':'register_date'})
registered.nunique()

device_id        78310
register_date       91
dtype: int64

In [90]:
df = case_data.merge(registered, on='device_id', how='left')

In [91]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month_index,register_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1,NaT
...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,3,2020-03-31
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,3,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,3,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,3,2020-03-31


In [82]:
case_data.groupby('event', as_index=False).agg(users=('device_id', 'nunique')).sort_values('users', ascending=False)

,event,users
1,app_start,190884
5,search,184488
2,choose_item,155691
0,app_install,154597
6,tap_basket,125414
4,register,78310
3,purchase,70875


In [92]:
df.loc[df.register_date < df.date].groupby('event', as_index=False).agg(users=('device_id', 'nunique')).sort_values('users', ascending=False)

,event,users
0,app_start,40991
3,search,40482
1,choose_item,37926
4,tap_basket,34517
2,purchase,24880


In [93]:
df.loc[df.register_date >= df.date].groupby('event', as_index=False).agg(users=('device_id', 'nunique')).sort_values('users', ascending=False)

,event,users
1,app_start,78310
2,choose_item,78310
4,register,78310
5,search,78310
6,tap_basket,78310
3,purchase,67753
0,app_install,63526


In [100]:
# Conversion into first purchase by channel
cr_by_channel = case_data[case_data.event.isin(['app_start', 'purchase'])] \
    .pivot_table(index='utm_source', columns='event', values='device_id', aggfunc='nunique')
cr_by_channel['cr'] = cr_by_channel['purchase'] / cr_by_channel['app_start']

In [101]:
cr_by_channel

event,app_start,purchase,cr
utm_source,,,
-,52268,16598,0.317556
facebook_ads,25957,9017,0.347382
google_ads,38092,11339,0.297674
instagram_ads,31045,10762,0.346658
referal,15924,6362,0.399523
vk_ads,34079,12364,0.362804
yandex-direct,40707,12028,0.295477


In [105]:
# Median value of first purchase by channel
first_purchases = case_data \
    .loc[case_data.event == 'purchase'][['date', 'purchase_sum', 'device_id', 'utm_source']] \
    .sort_values('date') \
    .drop_duplicates(subset='device_id')
first_purchases

,date,purchase_sum,device_id,utm_source
7691,2020-01-01,265.5,6081155,-
8289,2020-01-01,292.0,1955306,yandex-direct
8290,2020-01-01,664.0,20176327,facebook_ads
8291,2020-01-01,337.0,28603960,-
8292,2020-01-01,502.0,14028316,referal
...,...,...,...,...
2506842,2020-03-31,670.0,32122725,-
2506845,2020-03-31,407.0,23188703,instagram_ads
2506848,2020-03-31,149.5,25732558,referal
2506852,2020-03-31,442.5,17778508,referal


In [117]:
median_basket = first_purchases \
    .groupby('utm_source', as_index=False) \
    .agg(median_basket=('purchase_sum', 'median'), new_clients=('device_id', 'nunique')) \
    .sort_values('median_basket', ascending=False)
median_basket

,utm_source,median_basket,new_clients
0,-,398.5,14786
4,referal,395.5,5803
3,instagram_ads,393.5,9820
5,vk_ads,393.0,11460
6,yandex-direct,392.5,10936
2,google_ads,390.5,10167
1,facebook_ads,389.0,7903


In [130]:
df = case_data.groupby('utm_source', as_index=False).agg(revenue=('purchase_sum', 'sum'))
df = df.merge(median_basket[['utm_source', 'new_clients']], on='utm_source')
df['marketing_spend'] = pd.Series([0, 8590498,  10534878, 8561626, 0, 9553531, 10491707])
df.at[4, 'marketing_spend'] = df.loc[df.utm_source == 'referal']['new_clients']*2*100
df

,utm_source,revenue,new_clients,marketing_spend
0,-,21449749.5,14786,0
1,facebook_ads,12249901.0,7903,8590498
2,google_ads,12868276.0,10167,10534878
3,instagram_ads,14546969.0,9820,8561626
4,referal,8837044.5,5803,1160600
5,vk_ads,16389652.5,11460,9553531
6,yandex-direct,13915368.0,10936,10491707


In [132]:
df['romi'] = round((df.revenue - df.marketing_spend) /  df.marketing_spend * 100, 2)
df

,utm_source,revenue,new_clients,marketing_spend,romi
0,-,21449749.5,14786,0,inf
1,facebook_ads,12249901.0,7903,8590498,42.60
2,google_ads,12868276.0,10167,10534878,22.15
3,instagram_ads,14546969.0,9820,8561626,69.91
4,referal,8837044.5,5803,1160600,661.42
5,vk_ads,16389652.5,11460,9553531,71.56
6,yandex-direct,13915368.0,10936,10491707,32.63


In [ ]:
Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

In [103]:
case_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month_index
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


In [16]:
case_data.event.value_counts()

app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: event, dtype: int64

In [69]:
case_data.utm_source.value_counts()

-                610458
vk_ads           418456
yandex-direct    390232
instagram_ads    382891
google_ads       374257
facebook_ads     336953
referal          234721
Name: utm_source, dtype: int64